In [1]:
%matplotlib inline
import os,random

os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["KERAS_FLAGS"]  = "device=gpu%d"%(0)

import numpy as np
from keras.utils import np_utils
import keras.models as models
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D, ZeroPadding3D ,Conv3D
from keras.regularizers import *
from keras.optimizers import adam
import matplotlib.pyplot as plt
import seaborn as sns
import cPickle, random, sys, keras
import missinglink

Using TensorFlow backend.


In [2]:
import h5py
filename = '../data_cwt.hdf5'
f = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % f.keys())


Keys: [u'Classes', u'LabelClass', u'LabelSnr', u'Snrs', u'X_samples', u'Y_samples']


In [3]:
X = f['X_samples']
Y = f['Y_samples']


In [4]:
np.random.seed(2016)
n_examples = X.shape[0]
n_train = n_examples * 0.5
train_idx = np.random.choice(range(0,int(n_examples)), size=int(n_train), replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))

In [5]:
def generate_data_from_hdf5(X_list,Y_list ,indexs ,batchSize):
    for i in xrange(0, len(X_list), batchsize):
        current_indexs = indexs[i:i+batchsize]
        yield (map(lambda t: X_list[t], current_indexs), map(lambda t: Y_list[t], current_indexs))


In [6]:
in_shp = list(X.shape[1:])
print X.shape, in_shp
classes = f['Classes']

(220000, 224, 224, 3) [224, 224, 3]


In [7]:
print type(in_shp)

<type 'list'>


In [8]:
# Build VT-CNN2 Neural Net model using Keras primitives -- 
#  - Reshape [N,2,128] to [N,1,2,128] on input
#  - Pass through 2 2DConv/ReLu layers
#  - Pass through 2 Dense layers (ReLu and Softmax)
#  - Perform categorical cross entropy optimization

dr = 0.5 # dropout rate (%)
model = models.Sequential()
model.add(Reshape(in_shp+[1], input_shape=in_shp))
model.add(ZeroPadding3D((0,2,2)))
model.add(Conv3D(256,(1,3,3),kernel_initializer="glorot_uniform", name="conv1", activation="relu", padding="valid"))
model.add(Dropout(dr))
model.add(ZeroPadding3D((0, 2,2)))
model.add(Conv3D(80,(2,3,3),padding="valid", activation="relu", name="conv2", kernel_initializer='glorot_uniform'))
model.add(Dropout(dr))
model.add(Flatten())
model.add(Dense(256, kernel_initializer="he_normal", activation="relu", name="dense1"))
model.add(Dropout(dr))
model.add(Dense(11, kernel_initializer="he_normal", name="dense2"))
model.add(Activation('softmax'))
model.add(Reshape([len(classes)]))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 224, 224, 3, 1)    0         
_________________________________________________________________
zero_padding3d_1 (ZeroPaddin (None, 224, 228, 7, 1)    0         
_________________________________________________________________
conv1 (Conv3D)               (None, 224, 226, 5, 256)  2560      
_________________________________________________________________
dropout_1 (Dropout)          (None, 224, 226, 5, 256)  0         
_________________________________________________________________
zero_padding3d_2 (ZeroPaddin (None, 224, 230, 9, 256)  0         
_________________________________________________________________
conv2 (Conv3D)               (None, 223, 228, 7, 80)   368720    
_________________________________________________________________
dropout_2 (Dropout)          (None, 223, 228, 7, 80)   0         
__________

In [9]:
# Set up some params 
nb_epoch = 100     # number of epochs to train on
batch_size = 200  # training batch size
steps_per_epoch = (X.shape[0]/batch_size)/2 # data is split to training and testing
print steps_per_epoch

550


In [ ]:
# perform training ...
missinglink_callback = missinglink.KerasCallback(owner_id="73b7dbec-273d-c6b7-776d-55812449a4e4", project_token="WxqnIeHhwiLIFejy")
missinglink_callback.set_properties(display_name='cwt experiment', description='local, simple conv net')
#   - call the main training loop in keras for our network+dataset
filepath = 'convmodrecnets_CNN2_0.5.wts.h5'
history = model.fit_generator(
    generate_data_from_hdf5(X,Y ,train_idx ,batch_size),
    validation_data=generate_data_from_hdf5(X,Y ,test_idx ,batch_size),
    nb_epoch=nb_epoch,
    steps_per_epoch=steps_per_epoch,
    validation_steps=steps_per_epoch,
    verbose=2,
    callbacks = [
        keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto'),
        keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)
    ])
# we re-load the best weights once training is finished
model.load_weights(filepath)

/home/yoni/repos/university/ENV/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  app.launch_new_instance()
/home/yoni/repos/university/ENV/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., verbose=2, validation_data=<generator..., steps_per_epoch=550, epochs=100, callbacks=[<keras.ca..., validation_steps=550)`
  app.launch_new_instance()
